# 2016


In [104]:
import pandas as pd
import re

In [286]:
def extract_rows(year_df, df_base, bio_map, region_idx, year):
    rows = []
    
    for i in range(3, len(year_df.columns) - 1):
        col_dosis, col_cobertura = year_df.columns[i], year_df.columns[i + 1]
        nombre_columna = col_dosis[0].replace("\n", " ").strip()

        if "POBLACIÓN" in nombre_columna.upper() or "POBLACION" in nombre_columna.upper() or "TOTAL" in nombre_columna.upper():
            continue  # ignorar columnas de población o totales

        
        if nombre_columna not in bio_map:
            print(nombre_columna)

        if nombre_columna not in bio_map or "%" not in col_cobertura[1]:
            continue  # ignorar columnas no mapeadas o sin cobertura válida
            
        
        biologico, grupo_poblacional = bio_map[nombre_columna]
        tipo_dosis = col_dosis[1].strip()
        

        for idx, row in df_base.iterrows():
            rows.append({
                "ANO": year,
                "CODIGO DEPARTAMENTO": row["CODIGO DEPARTAMENTO"],
                "DEPARTAMENTO": row["DEPARTAMENTO"],
                "META": row["META"],
                "BIOLOGICO": biologico,
                "GRUPO POBLACIONAL": grupo_poblacional,
                "DOSIS APLICADAS": year_df.iloc[idx, i],
                "COBERTURA": year_df.iloc[idx, i+1],
                "TIPO DOSIS": tipo_dosis,
                "REGION": year_df.iloc[idx, region_idx],
            })
        


    df_curado = pd.DataFrame(rows)

    df_curado = df_curado[df_curado['DEPARTAMENTO'] != "TOTAL"]
    df_curado.replace({"REGION": {"LLANOS", "ORINOQUIA"}}, inplace=True)
    df_curado['MUNICIPIO'] = None
    nuevo_orden = ['ANO', 'CODIGO DEPARTAMENTO', 'DEPARTAMENTO', 'MUNICIPIO', 
                'GRUPO POBLACIONAL', 'BIOLOGICO', 'META', 'DOSIS APLICADAS',
                    'COBERTURA', 'TIPO DOSIS', 'REGION']


    df_curado = df_curado[nuevo_orden]
    return df_curado


In [239]:
archivo = "Cobertura_Vacunacion_Departamental_2016-2019.xlsx"
df_2016 = pd.read_excel(archivo, sheet_name="2016", header=[0, 1])

df_base = df_2016.iloc[:, 0:3].copy()
df_base.columns = ["CODIGO DEPARTAMENTO", "DEPARTAMENTO", "META"]


mapa_biologico = {
    "BCG RECIEN NACIDO (HASTA 29 DÍAS) + MENOR UN AÑO (A PARTIR DE 30 DIAS HASTA 11 MESES 29 DIAS)": ("BCG", "RECIÉN NACIDO HASTA 11 MESES"),
    "HB RECIEN NACIDO (HASTA 29 DÍAS)": ("HB", "RECIÉN NACIDO"),
    "ANTIPOLIO INACTIVADA (VIP) PARENTERAL <  DE 1 AÑO": ("ANTIPOLIO INACTIVADA", "<1 AÑO"),
    "VOP <DE1 AÑO + VIP <1AÑO 3as": ("VOP+VIP", "<1 AÑO 3as"),
    "PENTA MENOR DE UN AÑO": ("PENTA", "MENOR DE 1 AÑO"),
    "ROTAVIRUS DE 2 A 3 MESES Y 21 DIAS": ("ROTAVIRUS", "2 A 3 MESES Y 21 DÍAS"),
    "ROTAVIRUS DE 4 A 11 MESES Y 29 DIAS": ("ROTAVIRUS", "4 A 11 MESES Y 29 DÍAS"),
    "FLU DE 6 MESES A 11 MESES": ("FLU", "6 A 11 MESES"),
    "Neumo (2 a 11 Meses 29 Dias)": ("NEUMO", "2 A 11 MESES 29 DÍAS"),
    "SRP (T.V.) DE UN AÑO": ("SRP", "DE UN AÑO"),
    "VARICELA DE 1 AÑO 1a": ("VARICELA", "1 AÑO"),
    "F.A. 1 AÑO": ("F.A.", "1 AÑO"),
    "Neumo de (12 meses a 23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (12-23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (24- 35 Meses)": ("NEUMO", "24 A 35 MESES"),
    "HA 1 AÑO": ("HA", "1 AÑO"),
    "VOP Al año de la 3ra Dosis": ("VOP", "AL AÑO DE 3ra DOSIS"),
    "DPT Al año de la 3ra Dosis": ("DPT", "AL AÑO DE 3ra DOSIS"),
    "VOP 5 AÑOS": ("VOP", "5 AÑOS"),
    "DPT 5 AÑOS": ("DPT", "5 AÑOS"),
    "SRP (T.V.) 5 AÑOS": ("SRP", "5 AÑOS"),
    "FLU de 19 a 49 años ÚNICA": ("FLU", "19 A 49 AÑOS"),
    "FLU de 50 años y más": ("FLU", "50 AÑOS Y MÁS"),
    "Gestantes a partir de la semana 14": ("GESTANTES", "SEMANA 14+"),
    "TdaP ACELULAR GESTANTES DE 10 AÑOS Y MAS": ("TDAP ACELULAR", "GESTANTES ≥10 AÑOS")
}

# Crear filas estructuradas
df_curado = extract_rows(df_2016, df_base, mapa_biologico, 60, 2016)
df_curado.sample(15)

,ANO,CODIGO DEPARTAMENTO,DEPARTAMENTO,MUNICIPIO,GRUPO POBLACIONAL,BIOLOGICO,META,DOSIS APLICADAS,COBERTURA,TIPO DOSIS,REGION
87,2016,85.0,CASANARE,None,<1 AÑO,ANTIPOLIO INACTIVADA,7580,6836,90.2,1ras. Dosis,LLANOS
395,2016,23.0,CORDOBA,None,5 AÑOS,DPT,32530,29134,90.3,2do. Ref.,CARIBE
420,2016,81.0,ARAUCA,None,5 AÑOS,SRP,4272,4522,92.2,Ref.,LLANOS
130,2016,25.0,CUNDINAMARCA,None,<1 AÑO 3as,VOP+VIP,37799,32219,85.2,3.Dosis,ANDINA
407,2016,63.0,QUINDIO,None,5 AÑOS,DPT,5813,5973,92.2,2do. Ref.,ANDINA
424,2016,13.0,BOLIVAR,None,5 AÑOS,SRP,22708,19148,85.2,Ref.,CARIBE
335,2016,70.0,SUCRE,None,2 A 11 MESES 29 DÍAS,NEUMO,16298,14353,88.1,1ra Dosis,CARIBE
221,2016,70.0,SUCRE,None,2 A 3 MESES Y 21 DÍAS,ROTAVIRUS,16298,14107,86.6,1ra Dosis,CARIBE
187,2016,97.0,VAUPES,None,MENOR DE 1 AÑO,PENTA,861,744,86.4,1ras. Dosis,AMAZONIA
326,2016,47001.0,SANTA MARTA,None,2 A 11 MESES 29 DÍAS,NEUMO,9061,8579,94.7,1ra Dosis,CARIBE


In [240]:
df_curado['BIOLOGICO'].unique()

array(['BCG', 'HB', 'ANTIPOLIO INACTIVADA', 'VOP+VIP', 'PENTA',
       'ROTAVIRUS', 'FLU', 'NEUMO', 'VOP', 'DPT', 'SRP'], dtype=object)

In [241]:
df_curado['GRUPO POBLACIONAL'].unique()

array(['RECIÉN NACIDO HASTA 11 MESES', 'RECIÉN NACIDO', '<1 AÑO',
       '<1 AÑO 3as', 'MENOR DE 1 AÑO', '2 A 3 MESES Y 21 DÍAS',
       '4 A 11 MESES Y 29 DÍAS', '6 A 11 MESES', '2 A 11 MESES 29 DÍAS',
       '5 AÑOS'], dtype=object)

In [242]:
df_curado['TIPO DOSIS'].unique()

array(['Dosis Unica', 'D. Unica', '1ras. Dosis', '3.Dosis', '1ra Dosis',
       '2da Dosis', '2do. Ref.', 'Ref.'], dtype=object)

In [243]:
df_curado.to_csv("cobertura_vacunacion_2016.csv", index=False)

# 2017

In [281]:
df_2017 = pd.read_excel(archivo, sheet_name="2017", header=[0, 1])


df_base = df_2017.iloc[:, 0:3].copy()
df_base.columns = ["CODIGO DEPARTAMENTO", "DEPARTAMENTO", "META"]


mapa_biologico = {
    "BCG RECIEN NACIDO (HASTA 29 DÍAS) + MENOR UN AÑO (A PARTIR DE 30 DIAS HASTA 11 MESES 29 DIAS)": ("BCG", "RECIÉN NACIDO HASTA 11 MESES"),
    "HB RECIEN NACIDO (HASTA 29 DÍAS) (Primeras 12 Horas)+ Despues de las primeras 12 horas": ("HB", "RECIÉN NACIDO"),
    "ANTIPOLIO INACTIVADA (VIP) PARENTERAL <  DE 1 AÑO": ("ANTIPOLIO INACTIVADA", "<1 AÑO"),
    "VOP <DE1 AÑO + VIP <1AÑO 3as": ("VOP+VIP", "<1 AÑO 3as"),
    "PENTA MENOR DE UN AÑO": ("PENTA", "MENOR DE 1 AÑO"),
    "ROTAVIRUS DE 2 A 3 MESES Y 21 DIAS": ("ROTAVIRUS", "2 A 3 MESES Y 21 DÍAS"),
    "ROTAVIRUS DE 4 A 11 MESES Y 29 DIAS": ("ROTAVIRUS", "4 A 11 MESES Y 29 DÍAS"),
    "FLU DE 6 MESES A 11 MESES": ("FLU", "6 A 11 MESES"),
    "VARICELA DE 1 AÑO 1a": ("VARICELA", "1 AÑO"),
    "F.A. 1 AÑO": ("F.A.", "1 AÑO"),
    "Neumo (2 a 11 Meses 29 Dias)": ("NEUMO", "2 A 11 MESES 29 DÍAS"),
    "Neumo de (12 meses a 23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (12-23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (24- 35 Meses)": ("NEUMO", "24 A 35 MESES"),
    "HA 1 AÑO": ("HA", "1 AÑO"),
    "VOP Al año de la 3ra Dosis": ("VOP", "AL AÑO DE 3ra DOSIS"),
    "DPT Al año de la 3ra Dosis": ("DPT", "AL AÑO DE 3ra DOSIS"),
    "VOP 5 AÑOS": ("VOP", "5 AÑOS"),
    "DPT 5 AÑOS": ("DPT", "5 AÑOS"),
    "SRP (T.V.) DE UN AÑO": ("SRP", "DE UN AÑO"),
    "SRP (T.V.) 5 AÑOS": ("SRP", "5 AÑOS"),
    "FLU de 19 a 49 años ÚNICA": ("FLU", "19 A 49 AÑOS"),
    "FLU de 50 años y más": ("FLU", "50 AÑOS Y MÁS"),
    "Gestantes a partir de la semana 14": ("GESTANTES", "SEMANA 14+"),
    "TdaP ACELULAR GESTANTES DE 10 AÑOS Y MAS": ("TDAP ACELULAR", "GESTANTES ≥10 AÑOS"),
    "VPH": ("VPH", "9 AÑOS"),
}

# Crear filas estructuradas
df_curado = extract_rows(df_2017, df_base, mapa_biologico, 74, 2017)


df_curado.sample(15)

,ANO,CODIGO DEPARTAMENTO,DEPARTAMENTO,MUNICIPIO,GRUPO POBLACIONAL,BIOLOGICO,META,DOSIS APLICADAS,COBERTURA,TIPO DOSIS,REGION
308,2017,97.0,VAUPES,None,2 A 3 MESES Y 21 DÍAS,ROTAVIRUS,831.0,568.0,68.4,1ra Dosis,AMAZONIA
783,2017,8.0,ATLANTICO,None,AL AÑO DE 3ra DOSIS,VOP,20423.0,19633.0,94.2,1er. Ref.,CARIBE
401,2017,85.0,CASANARE,None,6 A 11 MESES,FLU,7346.0,4360.0,79.1,2da Dosis,LLANOS
1067,2017,27.0,CHOCO,None,SEMANA 14+,GESTANTES,11015.0,5691.0,NaN,UNICA,PACIFICO
910,2017,20.0,CESAR,None,5 AÑOS,DPT,23054.0,22492.0,93.4,2do. Ref.,CARIBE
143,2017,86.0,PUTUMAYO,None,<1 AÑO 3as,VOP+VIP,4978.0,4592.0,92.2,3.Dosis,AMAZONIA
953,2017,94.0,GUAINIA,None,5 AÑOS,SRP,833.0,758.0,81.4,Ref.,AMAZONIA
844,2017,54.0,NORTE DE SANTANDER,None,AL AÑO DE 3ra DOSIS,DPT,23615.0,19996.0,83.3,1er. Ref.,ANDINA
209,2017,27.0,CHOCO,None,MENOR DE 1 AÑO,PENTA,11015.0,9323.0,84.6,2das. Dosis,PACIFICO
964,2017,66.0,RISARALDA,None,5 AÑOS,SRP,11159.0,10840.0,91.3,Ref.,ANDINA


In [282]:
df_curado['BIOLOGICO'].unique()

array(['BCG', 'HB', 'ANTIPOLIO INACTIVADA', 'VOP+VIP', 'PENTA',
       'ROTAVIRUS', 'FLU', 'NEUMO', 'SRP', 'VARICELA', 'F.A.', 'HA',
       'VOP', 'DPT', 'GESTANTES', 'TDAP ACELULAR', 'VPH'], dtype=object)

In [283]:
df_curado['GRUPO POBLACIONAL'].unique()

array(['RECIÉN NACIDO HASTA 11 MESES', 'RECIÉN NACIDO', '<1 AÑO',
       '<1 AÑO 3as', 'MENOR DE 1 AÑO', '2 A 3 MESES Y 21 DÍAS',
       '4 A 11 MESES Y 29 DÍAS', '6 A 11 MESES', '2 A 11 MESES 29 DÍAS',
       'DE UN AÑO', '1 AÑO', '12 A 23 MESES', '24 A 35 MESES',
       'AL AÑO DE 3ra DOSIS', '5 AÑOS', '19 A 49 AÑOS', '50 AÑOS Y MÁS',
       'SEMANA 14+', 'GESTANTES ≥10 AÑOS', '9 AÑOS'], dtype=object)

In [284]:
df_curado['TIPO DOSIS'].unique()

array(['Dosis Unica', 'D. Unica', '1ras. Dosis', '3.Dosis', '2das. Dosis',
       '3ras Dosis', '1ra Dosis', '2da Dosis', 'U.Dósis', 'REF.',
       'U. Dósis', '1er. Ref.', '2do. Ref.', 'Ref.', 'UNICA',
       'UNICA DOSIS EN CADA EMBARAZO', '1a', '2a', '3a'], dtype=object)

In [285]:
df_curado.to_csv("cobertura_vacunacion_2017.csv", index=False)

# 2018

In [289]:
df_2018 = pd.read_excel(archivo, sheet_name="2018", header=[0, 1])


df_base = df_2018.iloc[:, 0:3].copy()
df_base.columns = ["CODIGO DEPARTAMENTO", "DEPARTAMENTO", "META"]


mapa_biologico = {
    "BCG RECIEN NACIDO (HASTA 29 DÍAS) + MENOR UN AÑO (A PARTIR DE 30 DIAS HASTA 11 MESES 29 DIAS)": ("BCG", "RECIÉN NACIDO HASTA 11 MESES"),
    "HB RECIEN NACIDO (HASTA 29 DÍAS) (Primeras 12 Horas)+ Despues de las primeras 12 horas": ("HB", "RECIÉN NACIDO"),
    "ANTIPOLIO INACTIVADA (VIP) PARENTERAL <  DE 1 AÑO": ("ANTIPOLIO INACTIVADA", "<1 AÑO"),
    "VOP <DE1 AÑO + VIP <1AÑO 3as": ("VOP+VIP", "<1 AÑO 3as"),
    "PENTA MENOR DE UN AÑO": ("PENTA", "MENOR DE 1 AÑO"),
    "ROTAVIRUS DE 2 A 3 MESES Y 21 DIAS": ("ROTAVIRUS", "2 A 3 MESES Y 21 DÍAS"),
    "ROTAVIRUS DE 4 A 11 MESES Y 29 DIAS": ("ROTAVIRUS", "4 A 11 MESES Y 29 DÍAS"),
    "FLU DE 6 MESES A 11 MESES": ("FLU", "6 A 11 MESES"),
    "VARICELA DE 1 AÑO 1a": ("VARICELA", "1 AÑO"),
    "F.A. 1 AÑO": ("F.A.", "1 AÑO"),
    "Neumo (2 a 11 Meses 29 Dias)": ("NEUMO", "2 A 11 MESES 29 DÍAS"),
    "Neumo de (12 meses a 23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (12-23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (24- 35 Meses)": ("NEUMO", "24 A 35 MESES"),
    "HA 1 AÑO": ("HA", "1 AÑO"),
    "VOP Al año de la 3ra Dosis": ("VOP", "AL AÑO DE 3ra DOSIS"),
    "DPT Al año de la 3ra Dosis": ("DPT", "AL AÑO DE 3ra DOSIS"),
    "VOP 5 AÑOS": ("VOP", "5 AÑOS"),
    "DPT 5 AÑOS": ("DPT", "5 AÑOS"),
    "SRP (T.V.) DE UN AÑO": ("SRP", "DE UN AÑO"),
    "SRP (T.V.) 5 AÑOS": ("SRP", "5 AÑOS"),
    "FLU de 19 a 49 años ÚNICA": ("FLU", "19 A 49 AÑOS"),
    "FLU de 50 años y más": ("FLU", "50 AÑOS Y MÁS"),
    "Gestantes a partir de la semana 14": ("GESTANTES", "SEMANA 14+"),
    "TdaP ACELULAR GESTANTES DE 10 AÑOS Y MAS": ("TDAP ACELULAR", "GESTANTES ≥10 AÑOS"),
    "VPH": ("VPH", "9 AÑOS"),
}

# Crear filas estructuradas
df_curado = extract_rows(df_2018, df_base, mapa_biologico, 73, 2018)


df_curado.sample(15)

HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)


,ANO,CODIGO DEPARTAMENTO,DEPARTAMENTO,MUNICIPIO,GRUPO POBLACIONAL,BIOLOGICO,META,DOSIS APLICADAS,COBERTURA,TIPO DOSIS,REGION
877,2018,94.0,94.0,None,AL AÑO DE 3ra DOSIS,DPT,GUAINIA,649.0,73.40,1er. Ref.,AMAZONIA
828,2018,85.0,85.0,None,AL AÑO DE 3ra DOSIS,VOP,CASANARE,6613.0,89.20,1er. Ref.,LLANOS
247,2018,73.0,73.0,None,MENOR DE 1 AÑO,PENTA,TOLIMA,17205.0,94.10,2das. Dosis,ANDINA
150,2018,47.0,47.0,None,<1 AÑO 3as,VOP+VIP,MAGDALENA,16302.0,93.50,3.Dosis,CARIBE
142,2018,20.0,20.0,None,<1 AÑO 3as,VOP+VIP,CESAR,22560.0,97.30,3.Dosis,CARIBE
944,2018,NaN,NaN,None,5 AÑOS,VOP,NaN,NaN,NaN,2do. Ref.,NaN
984,2018,NaN,NaN,None,5 AÑOS,DPT,NaN,NaN,NaN,2do. Ref.,NaN
788,2018,27.0,27.0,None,1 AÑO,HA,CHOCO,10896.0,96.40,U. Dósis,PACIFICO
1047,2018,23.0,23.0,None,19 A 49 AÑOS,FLU,CORDOBA,11689.0,NaN,UNICA,CARIBE
457,2018,63.0,63.0,None,6 A 11 MESES,FLU,QUINDIO,4573.0,102.80,2da Dosis,ANDINA


In [290]:
df_curado['BIOLOGICO'].unique()

array(['BCG', 'HB', 'ANTIPOLIO INACTIVADA', 'VOP+VIP', 'PENTA',
       'ROTAVIRUS', 'FLU', 'NEUMO', 'SRP', 'VARICELA', 'F.A.', 'HA',
       'VOP', 'DPT', 'GESTANTES', 'TDAP ACELULAR', 'VPH'], dtype=object)

In [291]:
df_curado['GRUPO POBLACIONAL'].unique()

array(['RECIÉN NACIDO HASTA 11 MESES', 'RECIÉN NACIDO', '<1 AÑO',
       '<1 AÑO 3as', 'MENOR DE 1 AÑO', '2 A 3 MESES Y 21 DÍAS',
       '4 A 11 MESES Y 29 DÍAS', '6 A 11 MESES', '2 A 11 MESES 29 DÍAS',
       'DE UN AÑO', '1 AÑO', '12 A 23 MESES', 'AL AÑO DE 3ra DOSIS',
       '5 AÑOS', '19 A 49 AÑOS', '50 AÑOS Y MÁS', 'SEMANA 14+',
       'GESTANTES ≥10 AÑOS', '9 AÑOS'], dtype=object)

In [292]:
df_curado['TIPO DOSIS'].unique()

array(['Dosis Unica', 'D. Unica', '1ras. Dosis', '3.Dosis', '2das. Dosis',
       '3ras .Dosis', '1ra Dosis', '2da Dosis', 'U.Dósis', 'REF.',
       'U. Dósis', '1er. Ref.', '2do. Ref.', 'Ref.', 'UNICA',
       'UNICA DOSIS EN CADA EMBARAZO', '1a', '2a'], dtype=object)

In [293]:
df_curado.to_csv("cobertura_vacunacion_2018.csv", index=False)

In [ ]:
df_2018 = pd.read_excel(archivo, sheet_name="2018", header=[0, 1])


df_base = df_2018.iloc[:, 0:3].copy()
df_base.columns = ["CODIGO DEPARTAMENTO", "DEPARTAMENTO", "META"]


mapa_biologico = {
    "BCG RECIEN NACIDO (HASTA 29 DÍAS) + MENOR UN AÑO (A PARTIR DE 30 DIAS HASTA 11 MESES 29 DIAS)": ("BCG", "RECIÉN NACIDO HASTA 11 MESES"),
    "HB RECIEN NACIDO (HASTA 29 DÍAS) (Primeras 12 Horas)+ Despues de las primeras 12 horas": ("HB", "RECIÉN NACIDO"),
    "ANTIPOLIO INACTIVADA (VIP) PARENTERAL <  DE 1 AÑO": ("ANTIPOLIO INACTIVADA", "<1 AÑO"),
    "VOP <DE1 AÑO + VIP <1AÑO 3as": ("VOP+VIP", "<1 AÑO 3as"),
    "PENTA MENOR DE UN AÑO": ("PENTA", "MENOR DE 1 AÑO"),
    "ROTAVIRUS DE 2 A 3 MESES Y 21 DIAS": ("ROTAVIRUS", "2 A 3 MESES Y 21 DÍAS"),
    "ROTAVIRUS DE 4 A 11 MESES Y 29 DIAS": ("ROTAVIRUS", "4 A 11 MESES Y 29 DÍAS"),
    "FLU DE 6 MESES A 11 MESES": ("FLU", "6 A 11 MESES"),
    "VARICELA DE 1 AÑO 1a": ("VARICELA", "1 AÑO"),
    "F.A. 1 AÑO": ("F.A.", "1 AÑO"),
    "Neumo (2 a 11 Meses 29 Dias)": ("NEUMO", "2 A 11 MESES 29 DÍAS"),
    "Neumo de (12 meses a 23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (12-23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (24- 35 Meses)": ("NEUMO", "24 A 35 MESES"),
    "HA 1 AÑO": ("HA", "1 AÑO"),
    "VOP Al año de la 3ra Dosis": ("VOP", "AL AÑO DE 3ra DOSIS"),
    "DPT Al año de la 3ra Dosis": ("DPT", "AL AÑO DE 3ra DOSIS"),
    "VOP 5 AÑOS": ("VOP", "5 AÑOS"),
    "DPT 5 AÑOS": ("DPT", "5 AÑOS"),
    "SRP (T.V.) DE UN AÑO": ("SRP", "DE UN AÑO"),
    "SRP (T.V.) 5 AÑOS": ("SRP", "5 AÑOS"),
    "FLU de 19 a 49 años ÚNICA": ("FLU", "19 A 49 AÑOS"),
    "FLU de 50 años y más": ("FLU", "50 AÑOS Y MÁS"),
    "Gestantes a partir de la semana 14": ("GESTANTES", "SEMANA 14+"),
    "TdaP ACELULAR GESTANTES DE 10 AÑOS Y MAS": ("TDAP ACELULAR", "GESTANTES ≥10 AÑOS"),
    "VPH": ("VPH", "9 AÑOS"),
}

# Crear filas estructuradas
df_curado = extract_rows(df_2018, df_base, mapa_biologico, 73, 2018)


df_curado.sample(15)

HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)


,ANO,CODIGO DEPARTAMENTO,DEPARTAMENTO,MUNICIPIO,GRUPO POBLACIONAL,BIOLOGICO,META,DOSIS APLICADAS,COBERTURA,TIPO DOSIS,REGION
877,2018,94.0,94.0,None,AL AÑO DE 3ra DOSIS,DPT,GUAINIA,649.0,73.40,1er. Ref.,AMAZONIA
828,2018,85.0,85.0,None,AL AÑO DE 3ra DOSIS,VOP,CASANARE,6613.0,89.20,1er. Ref.,LLANOS
247,2018,73.0,73.0,None,MENOR DE 1 AÑO,PENTA,TOLIMA,17205.0,94.10,2das. Dosis,ANDINA
150,2018,47.0,47.0,None,<1 AÑO 3as,VOP+VIP,MAGDALENA,16302.0,93.50,3.Dosis,CARIBE
142,2018,20.0,20.0,None,<1 AÑO 3as,VOP+VIP,CESAR,22560.0,97.30,3.Dosis,CARIBE
944,2018,NaN,NaN,None,5 AÑOS,VOP,NaN,NaN,NaN,2do. Ref.,NaN
984,2018,NaN,NaN,None,5 AÑOS,DPT,NaN,NaN,NaN,2do. Ref.,NaN
788,2018,27.0,27.0,None,1 AÑO,HA,CHOCO,10896.0,96.40,U. Dósis,PACIFICO
1047,2018,23.0,23.0,None,19 A 49 AÑOS,FLU,CORDOBA,11689.0,NaN,UNICA,CARIBE
457,2018,63.0,63.0,None,6 A 11 MESES,FLU,QUINDIO,4573.0,102.80,2da Dosis,ANDINA


In [ ]:
df_curado['BIOLOGICO'].unique()

array(['BCG', 'HB', 'ANTIPOLIO INACTIVADA', 'VOP+VIP', 'PENTA',
       'ROTAVIRUS', 'FLU', 'NEUMO', 'SRP', 'VARICELA', 'F.A.', 'HA',
       'VOP', 'DPT', 'GESTANTES', 'TDAP ACELULAR', 'VPH'], dtype=object)

In [ ]:
df_curado['GRUPO POBLACIONAL'].unique()

array(['RECIÉN NACIDO HASTA 11 MESES', 'RECIÉN NACIDO', '<1 AÑO',
       '<1 AÑO 3as', 'MENOR DE 1 AÑO', '2 A 3 MESES Y 21 DÍAS',
       '4 A 11 MESES Y 29 DÍAS', '6 A 11 MESES', '2 A 11 MESES 29 DÍAS',
       'DE UN AÑO', '1 AÑO', '12 A 23 MESES', 'AL AÑO DE 3ra DOSIS',
       '5 AÑOS', '19 A 49 AÑOS', '50 AÑOS Y MÁS', 'SEMANA 14+',
       'GESTANTES ≥10 AÑOS', '9 AÑOS'], dtype=object)

In [ ]:
df_curado['TIPO DOSIS'].unique()

array(['Dosis Unica', 'D. Unica', '1ras. Dosis', '3.Dosis', '2das. Dosis',
       '3ras .Dosis', '1ra Dosis', '2da Dosis', 'U.Dósis', 'REF.',
       'U. Dósis', '1er. Ref.', '2do. Ref.', 'Ref.', 'UNICA',
       'UNICA DOSIS EN CADA EMBARAZO', '1a', '2a'], dtype=object)

In [ ]:
df_curado.to_csv("cobertura_vacunacion_2018.csv", index=False)

# 2019

In [295]:
df_2019 = pd.read_excel(archivo, sheet_name="2019", header=[0, 1])


df_base = df_2019.iloc[:, 0:3].copy()
df_base.columns = ["CODIGO DEPARTAMENTO", "DEPARTAMENTO", "META"]


mapa_biologico = {
    "BCG RECIEN NACIDO (HASTA 29 DÍAS) + MENOR UN AÑO (A PARTIR DE 30 DIAS HASTA 11 MESES 29 DIAS)": ("BCG", "RECIÉN NACIDO HASTA 11 MESES"),
    "HB RECIEN NACIDO (HASTA 29 DÍAS) (Primeras 12 Horas)+ Despues de las primeras 12 horas": ("HB", "RECIÉN NACIDO"),
    "ANTIPOLIO INACTIVADA (VIP) PARENTERAL <  DE 1 AÑO": ("ANTIPOLIO INACTIVADA", "<1 AÑO"),
    "VOP <DE1 AÑO + VIP <1AÑO 3as": ("VOP+VIP", "<1 AÑO 3as"),
    "PENTA MENOR DE UN AÑO": ("PENTA", "MENOR DE 1 AÑO"),
    "ROTAVIRUS DE 2 A 3 MESES Y 21 DIAS": ("ROTAVIRUS", "2 A 3 MESES Y 21 DÍAS"),
    "ROTAVIRUS DE 4 A 11 MESES Y 29 DIAS": ("ROTAVIRUS", "4 A 11 MESES Y 29 DÍAS"),
    "FLU DE 6 MESES A 11 MESES": ("FLU", "6 A 11 MESES"),
    "VARICELA DE 1 AÑO 1a": ("VARICELA", "1 AÑO"),
    "VARICELA DE 5 AÑOS": ("VARICELA", "5 AÑOS"),
    "F.A. 1 AÑO": ("F.A.", "1 AÑO"),
    "Neumo (2 a 11 Meses 29 Dias)": ("NEUMO", "2 A 11 MESES 29 DÍAS"),
    "Neumo de (12 meses a 23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (12-23 Meses)": ("NEUMO", "12 A 23 MESES"),
    "Neumo de (24- 35 Meses)": ("NEUMO", "24 A 35 MESES"),
    "HA 1 AÑO": ("HA", "1 AÑO"),
    "VOP Al año de la 3ra Dosis": ("VOP", "AL AÑO DE 3ra DOSIS"),
    "DPT Al año de la 3ra Dosis": ("DPT", "AL AÑO DE 3ra DOSIS"),
    "VOP 5 AÑOS": ("VOP", "5 AÑOS"),
    "DPT 5 AÑOS": ("DPT", "5 AÑOS"),
    "SRP (T.V.) DE UN AÑO": ("SRP", "DE UN AÑO"),
    "SRP (T.V.) 5 AÑOS": ("SRP", "5 AÑOS"),
    "FLU DE 18 A 40 AÑOS UNICA": ("FLU", "18 A 40 AÑOS"),
    "FLU DE 41 A 59 AÑOS UNICA": ("FLU", "41 A 59 AÑOS"),
    "FLU de 19 a 49 años ÚNICA": ("FLU", "19 A 49 AÑOS"),
    "FLU de 50 años y más": ("FLU", "50 AÑOS Y MÁS"),
    "FLU DE 60 AÑOS Y MAS": ("FLU", "60 AÑOS Y MÁS"),
    "FLU Gestantes a partir de la semana 14": ("GESTANTES", "SEMANA 14+"),
    "TdaP ACELULAR GESTANTES DE 10 AÑOS Y MAS": ("TDAP ACELULAR", "GESTANTES ≥10 AÑOS"),
    "VPH Niñas de 9 años": ("VPH", "9 AÑOS"),
}

# Crear filas estructuradas
df_curado = extract_rows(df_2019, df_base, mapa_biologico, 75, 2019)


df_curado.sample(15)

HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)
HEPATITIS B RECIEN NACIDO (HASTA 29 DÍAS) RECIEN NACIDO (Despues de las Primeras 12 Horas)


,ANO,CODIGO DEPARTAMENTO,DEPARTAMENTO,MUNICIPIO,GRUPO POBLACIONAL,BIOLOGICO,META,DOSIS APLICADAS,COBERTURA,TIPO DOSIS,REGION
457,2019,66.0,RISARALDA,None,2 A 11 MESES 29 DÍAS,NEUMO,10258.0,10151.0,99.000000,1ra Dosis,ANDINA
1127,2019,97.0,VAUPES,None,GESTANTES ≥10 AÑOS,TDAP ACELULAR,789.0,670.0,84.900000,U. D. C/ embarazo,AMAZONIA
1038,2019,52.0,NARIÑO,None,60 AÑOS Y MÁS,FLU,20024.0,67421.0,108.566690,UNICA,PACIFICO
846,2019,63.0,QUINDIO,None,5 AÑOS,VOP,5600.0,5740.0,89.700000,2do. Ref.,ANDINA
40,2019,5.0,ANTIOQUIA,None,RECIÉN NACIDO,HB,74381.0,69229.0,93.100000,D. Unica,ANDINA
411,2019,47.0,MAGDALENA,None,6 A 11 MESES,FLU,16542.0,10276.0,62.100000,2da Dosis,CARIBE
907,2019,18.0,CAQUETA,None,5 AÑOS,SRP,7487.0,6896.0,86.000000,Dosis Ref.,AMAZONIA
540,2019,76.0,VALLE_DEL_CAUCA,None,DE UN AÑO,SRP,51369.0,52454.0,99.100000,U.Dósis,PACIFICO
1134,2019,8.0,ATLANTICO,None,9 AÑOS,VPH,19832.0,2115.0,22.715068,VPH 1a Dosis,CARIBE
21,2019,47.0,MAGDALENA,None,RECIÉN NACIDO HASTA 11 MESES,BCG,16542.0,12483.0,75.500000,Dosis Unica,CARIBE


In [296]:
df_curado['BIOLOGICO'].unique()

array(['BCG', 'HB', 'ANTIPOLIO INACTIVADA', 'VOP+VIP', 'PENTA',
       'ROTAVIRUS', 'FLU', 'NEUMO', 'SRP', 'VARICELA', 'HA', 'F.A.',
       'VOP', 'DPT', 'GESTANTES', 'TDAP ACELULAR', 'VPH'], dtype=object)

In [297]:
df_curado['GRUPO POBLACIONAL'].unique()

array(['RECIÉN NACIDO HASTA 11 MESES', 'RECIÉN NACIDO', '<1 AÑO',
       '<1 AÑO 3as', 'MENOR DE 1 AÑO', '2 A 3 MESES Y 21 DÍAS',
       '4 A 11 MESES Y 29 DÍAS', '6 A 11 MESES', '2 A 11 MESES 29 DÍAS',
       'DE UN AÑO', '1 AÑO', '12 A 23 MESES', 'AL AÑO DE 3ra DOSIS',
       '5 AÑOS', '41 A 59 AÑOS', '60 AÑOS Y MÁS', 'SEMANA 14+',
       'GESTANTES ≥10 AÑOS', '9 AÑOS'], dtype=object)

In [298]:
df_curado['TIPO DOSIS'].unique()

array(['Dosis Unica', 'D. Unica', '1ras. Dosis', '3.Dosis', '2das. Dosis',
       '3ras. Dosis', '1ra Dosis', '2da Dosis', 'U.Dósis', 'U. Dósis',
       'REF.', '1er. Ref.', '2do. Ref.', 'Dosis Ref.', 'UNICA',
       'U. D. C/ embarazo', 'VPH 1a Dosis', 'VPH 2a Dosis'], dtype=object)

In [299]:
df_curado.to_csv("cobertura_vacunacion_2019.csv", index=False)